## 모델링을 위한 전처리

In [19]:
import warnings
from sklearn.preprocessing import OneHotEncoder
from utils import info

warnings.filterwarnings('ignore')

### 시간 데이터 전처리

In [2]:
# 동일한 쿼리지만 동작 속도가 더 빠른 첫번째 쿼리문 사용
extract_query = """
SELECT
    *,
    EXTRACT(MONTH FROM pickup_hour) AS month,
    EXTRACT(DAY FROM pickup_hour) AS day,
    CAST(format_datetime('%u', pickup_hour) AS INT64) -1 AS weekday, # Python과 맞추기 위해 -1
    EXTRACT(HOUR FROM pickup_hour) AS hour,
    CASE WHEN CAST(FORMAT_DATETIME('%u', pickup_hour) AS INT64) IN (6, 7) THEN 1 ELSE 0 END AS is_weekend
FROM (
    SELECT 
        DATETIME_TRUNC(pickup_datetime, hour) AS pickup_hour,
        count(*) AS cnt
    FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2015` 
    WHERE EXTRACT(MONTH from pickup_datetime) = 1
    GROUP BY pickup_hour
)
ORDER BY pickup_hour
"""

# extract_query = """
# SELECT 
#     DATETIME_TRUNC(pickup_datetime, hour) AS pickup_hour,
#     EXTRACT(MONTH FROM pickup_datetime) AS month,
#     EXTRACT(DAY FROM pickup_datetime) AS day,
#     CAST(FORMAT_DATETIME('%u', pickup_datetime) AS INT64) -1 AS weekday, # Python과 맞추기 위해 -1
#     EXTRACT(HOUR FROM pickup_datetime) AS hour,
#     CASE WHEN CAST(FORMAT_DATETIME('%u', pickup_datetime) AS INT64) IN (6, 7) THEN 1 ELSE 0 END AS is_weekend,
#     count(*) AS cnt
# FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2015` 
# WHERE EXTRACT(MONTH from pickup_datetime) = 1
# GROUP BY pickup_hour, month, day, weekday, hour, is_weekend
# ORDER BY pickup_hour
# """

df = pd.read_gbq(query=extract_query, dialect='standard', project_id=info['project'])

In [3]:
df.tail(2)

,pickup_hour,cnt,month,day,weekday,hour,is_weekend
742,2015-01-31 22:00:00,29862,1,31,5,22,1
743,2015-01-31 23:00:00,29856,1,31,5,23,1


### BigQuery GIS로 Reverse Geocoding
- 좌표를 zip_code로 변환

In [33]:
base_query = """
WITH base_data AS 
(
    SELECT nyc_taxi.*, gis.* EXCEPT (zip_code_geom)
    FROM (
        SELECT *
        FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2015`
        WHERE 
            EXTRACT(MONTH from pickup_datetime) = 1
            and pickup_latitude  <= 90 and pickup_latitude >= -90
        ) AS nyc_taxi
    JOIN (
        SELECT zip_code, state_code, state_name, city, county, zip_code_geom
        FROM `bigquery-public-data.geo_us_boundaries.zip_codes`
        WHERE state_code='NY'
        ) AS gis 
    ON ST_CONTAINS(zip_code_geom, st_geogpoint(pickup_longitude, pickup_latitude))
)

SELECT 
    zip_code,
    DATETIME_TRUNC(pickup_datetime, hour) as pickup_hour,
    EXTRACT(MONTH FROM pickup_datetime) AS month,
    EXTRACT(DAY FROM pickup_datetime) AS day,
    CAST(format_datetime('%u', pickup_datetime) AS INT64) -1 AS weekday,
    EXTRACT(HOUR FROM pickup_datetime) AS hour,
    CASE WHEN CAST(FORMAT_DATETIME('%u', pickup_datetime) AS INT64) IN (6, 7) THEN 1 ELSE 0 END AS is_weekend,
    COUNT(*) AS cnt
FROM base_data 
GROUP BY zip_code, pickup_hour, month, day, weekday, hour, is_weekend
ORDER BY pickup_hour
"""

base_df = pd.read_gbq(query=base_query, dialect='standard', project_id=info['project'])

In [40]:
base_query = """
WITH base_data AS 
(
    SELECT nyc_taxi.*, gis.* EXCEPT (zip_code_geom)
    FROM (
        SELECT *
        FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2015`
        WHERE 
            EXTRACT(MONTH from pickup_datetime) = 1
            and pickup_latitude  <= 90 and pickup_latitude >= -90
        ) AS nyc_taxi
    JOIN (
        SELECT zip_code, state_code, state_name, city, county, zip_code_geom
        FROM `bigquery-public-data.geo_us_boundaries.zip_codes`
        WHERE state_code='NY'
        ) AS gis 
    ON ST_CONTAINS(zip_code_geom, st_geogpoint(pickup_longitude, pickup_latitude))
)

SELECT
    zip_code,
    pickup_hour,
    EXTRACT(MONTH FROM pickup_hour) AS month,
    EXTRACT(DAY FROM pickup_hour) AS day,
    CAST(format_datetime('%u', pickup_hour) AS INT64) -1 AS weekday,
    EXTRACT(HOUR FROM pickup_hour) AS hour,
    CASE WHEN CAST(FORMAT_DATETIME('%u', pickup_hour) AS INT64) IN (6, 7) THEN 1 ELSE 0 END AS is_weekend,
    cnt
FROM (
    SELECT 
        DATETIME_TRUNC(pickup_datetime, hour) AS pickup_hour,
        count(*) AS cnt,
        zip_code
    FROM base_data
    GROUP BY pickup_hour, zip_code
)
ORDER BY pickup_hour
"""

base_df = pd.read_gbq(query=base_query, dialect='standard', project_id=info['project'])

In [41]:
base_df.head()

,zip_code,pickup_hour,month,day,weekday,hour,is_weekend,cnt
0,11373,2015-01-01,1,1,3,0,0,14
1,11237,2015-01-01,1,1,3,0,0,38
2,11238,2015-01-01,1,1,3,0,0,95
3,10168,2015-01-01,1,1,3,0,0,32
4,10452,2015-01-01,1,1,3,0,0,5


### Categorical 데이터 전처리
- One-Hot Encoding

In [42]:
encoder = OneHotEncoder(handle_unknown='ignore') # 새로운 변수가 들어오면 error를 발생시키지 않고 모두 0으로 처리
encoder.fit(base_df[['zip_code']])

OneHotEncoder(handle_unknown='ignore')

In [43]:
ohe_output = encoder.transform(base_df[['zip_code']]).toarray()
oh_feature_df = pd.concat([base_df, pd.DataFrame(ohe_output, columns='zip_code_'+encoder.categories_[0])], axis=1)
oh_feature_df.head(3)

,zip_code,pickup_hour,month,day,weekday,hour,is_weekend,cnt,zip_code_10001,zip_code_10002,...,zip_code_12601,zip_code_12729,zip_code_12771,zip_code_13029,zip_code_13118,zip_code_13656,zip_code_13691,zip_code_14072,zip_code_14527,zip_code_14801
0,11373,2015-01-01,1,1,3,0,0,14,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,11237,2015-01-01,1,1,3,0,0,38,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,11238,2015-01-01,1,1,3,0,0,95,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Train and Test Split

In [44]:
def split_train_and_test(df, date):
    """
    Dataframe에서 train_df, test_df로 나눠주는 함수
    
    df : 시계열 데이터 프레임
    date : 기준점 날짜
    """
    train_df = df[df['pickup_hour'] < date]
    test_df = df[df['pickup_hour'] >= date]
    return train_df, test_df


def split_train_and_test_period(df, period):
    """
    Dataframe에서 train_df, test_df로 나눠주는 함수
    
    df : 시계열 데이터 프레임
    period : train/test 기준 일
    """
    criteria = str((max(df['pickup_hour']) - pd.Timedelta(days=period)).date())
    train_df = df[df['pickup_hour'] < criteria]
    test_df = df[df['pickup_hour'] >= criteria]
    return train_df, test_df

In [45]:
train_df, test_df = split_train_and_test(oh_feature_df, '2015-01-24') 

In [46]:
train_df, test_df = split_train_and_test_period(oh_feature_df, 7)

In [47]:
train_df.drop(columns=['zip_code', 'pickup_hour'], inplace=True)
test_df.drop(columns=['zip_code', 'pickup_hour'], inplace=True)

In [48]:
train_df.head(2)

,month,day,weekday,hour,is_weekend,cnt,zip_code_10001,zip_code_10002,zip_code_10003,zip_code_10004,...,zip_code_12601,zip_code_12729,zip_code_12771,zip_code_13029,zip_code_13118,zip_code_13656,zip_code_13691,zip_code_14072,zip_code_14527,zip_code_14801
0,1,1,3,0,0,14,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,3,0,0,38,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
y_train = train_df.pop('cnt')
y_test = test_df.pop('cnt')

x_train = train_df.copy()
x_test = test_df.copy()